# Importações + Declaração de funções (Apenas dê shift+enter)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster, FastMarkerCluster

In [0]:
LATITUDE = 'lat'
LONGITUDE = 'lon'
DATETIME = 'datetime'
TRAJ_ID = 'id'
TID = "tid"
UID = "user_id"

TILES = ["openstreetmap", "Mapbox Bright", "Mapbox Control Room", "Stamen Terrain", "Stamen Toner", "Stamen Watercolor",
         "CartoDB positron", "CartoDB dark_matter"]

def create_base_map(default_location, tile, default_zoom_start=12):
    """
    Generate a folium map.
    Parameters
    ----------
    default_location : tuple.
        Represents coordinates lat, lon which will be the center of the map.
    default_zoom_start : int, optional, default 12.
        Represents the zoom which will be the center of the map.
    tile : String.
        Represents the map's tiles.
    Returns
    -------
    base_map : folium.folium.Map.
        Represents a folium map.
    """
    base_map = folium.Map(location=default_location, control_scale=True,
                          zoom_start=default_zoom_start, tiles=tile)
    return base_map

def cluster(
    move_data,
    n_rows=None,
    lat_origin=None,
    lon_origin=None,
    zoom_start=12,
    base_map=None,
    tile=TILES[0],
    save_as_html=False,
    filename='cluster.html'
):
    """
    Generate visualization of Marker Cluster using folium plugin.
    Parameters
    ----------
    move_data : pymove.core.MoveDataFrameAbstract subclass.
        Input trajectory data.
    n_rows : int, optional, default None.
        Represents number of data rows that are will plot.
    lat_origin : float, optional, default None.
        Represents the latitude which will be the center of the map.
        If not entered, the first data from the dataset is used.
    lon_origin : float, optional, default None.
        Represents the longitude which will be the center of the map.
        If not entered, the first data from the dataset is used.
    zoom_start : int, optional, default 12.
        Represents the trajectory id.
    base_map : folium.folium.Map, optional, default None.
        Represents the folium map. If not informed, a new map is generated
        using the function create_base_map(), with the lat_origin, lon_origin
        and zoom_start.
    tile : String, optional, default 'OpenStreetMap'.
        Represents the map's tiles.
    save_as_html : bool, optional, default False.
        Represents if want save this visualization in a new file .html.
    filename : String, optional, default 'cluster.html'.
        Represents the file name of new file .html.
    Returns
    -------
    base_map : folium.folium.Map.
        Represents a folium map with visualization.
    """
    move_data = move_data.reset_index()

    if base_map is None:
        if lat_origin is None and lon_origin is None:
            lat_origin = move_data.loc[0][LATITUDE]
            lon_origin = move_data.loc[0][LONGITUDE]
        base_map = create_base_map(default_location=[lat_origin, lon_origin], tile=tile,
                                   default_zoom_start=zoom_start)

    if n_rows is None:
        n_rows = move_data.shape[0]

    mc = MarkerCluster()
    for row in move_data[:n_rows].iterrows():
        pop = "<b>Endereço:</b> " + str(row[1].endereco) \
              + "\n<b>Nome:</b> " + str(row[1].nome_fantasia) \
              + "\n<b>Email:</b> " + str(row[1].email) \
              + "\n<b>Telefone:</b> " + str(row[1].telefone_1)
        mc.add_child(folium.Marker(location=[row[1].lat, row[1].lon], popup=pop))
    base_map.add_child(mc)

    if save_as_html:
        base_map.save(outfile=filename)
    else:
        return base_map

def filter_by(data, bairro=None, cep=None, logradouro=None, complemento=None, 
          cnae_fiscal=None, data_inicio_atividade=None, 
          primeira_letra_nome_empresa=None, nome_empresa=None,
          cnpj=None, capital_social=None, porte=None,
          codigo_natureza_juridica=None, matriz_filial=None,
          motivo_situacao=None, razao_social=None, situacao=None):
    if codigo_natureza_juridica != None:  
        data = data[data['cod_nat_juridica'] == codigo_natureza_juridica].reset_index() # filtro por codigo da natureza juridica
    if bairro != None:  
        data = data[data['bairro'] == bairro].reset_index()
    if cep != None:  
        data = data[data['cep'] == cep].reset_index()
    if logradouro != None:
        data = data[data['logradouro'] == logradouro].reset_index()
    if complemento != None:  
        data = data[data['complemento'] == complemento].reset_index()
    if cnae_fiscal != None:
        data = data[data['cnae_fiscal'] == cnae_fiscal].reset_index()
    if data_inicio_atividade != None:  
        data = data[data['data_inicio_ativ'] >= data_inicio_atividade].reset_index()
    if cnpj != None:
        data = data[data['cnpj'] == cnpj].reset_index()
    # if conj_fiscal != None:  
    #     data = data[data['conj_fiscal'] == conj_fiscal]
    if capital_social != None:
        data = data[data['capital_social'] == capital_social].reset_index()
    if porte != None:  
        data = data[data['porte'] == porte].reset_index()
    if matriz_filial != None:
        data = data[data['matriz_filial'] == matriz_filial].reset_index()
    if motivo_situacao != None:
        data = data[data['motivo_situacao'] == motivo_situacao].reset_index()
    if razao_social != None:
        data = data[data['razao_social'] == razao_social].reset_index()
    if situacao != None:
        data = data[data['situacao'] == situacao].reset_index()
    if nome_empresa != None:
        data = data[data['nome_fantasia'] == nome_empresa].reset_index()

    print("Quantidade de dados: ", len(data))
    return data


# Leitura dos dados

In [10]:
path = "/content/drive/My Drive/CNPJ/data/dados_cnpj_fortaleza_enderecos.csv"

data = pd.read_csv(path)
# data =  pd.read_csv(open(path,'rU'), engine='c')
print("Quantidade de dados: ", len(data))

data = data[['capital_social', 'cnae_fiscal', 'cnpj', 'cod_nat_juridica', 'matriz_filial', 'motivo_situacao', 'nome_fantasia', 'opc_simples', 'porte', 'qualif_resp', 'razao_social', 'situacao', 'telefone_1', 'telefone_2', 'data_inicio_ativ', 'data_situacao', 'email', 'endereco', 'lat', 'lon']]
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Quantidade de dados:  514576


,capital_social,cnae_fiscal,cnpj,cod_nat_juridica,matriz_filial,motivo_situacao,nome_fantasia,opc_simples,porte,qualif_resp,razao_social,situacao,telefone_1,telefone_2,data_inicio_ativ,data_situacao,email,endereco,lat,lon
0,0.0,9430800,508605031,3999,1,71,0,0,5,16,ASSOCIACAO DOS ROTARIANOS DO ROTARY CLUB FORTA...,8,0,0,19970225,20081231,0,"RUA SENADOR POMPEU, 1539, CENTRO, FORTALEZA, C...",-3.733384,-38.531544
1,200000.0,8610101,991604972,2240,1,1,0,0,1,49,AMH ASSISTENCIA MEDICO HOSPITALAR S/C LTDA,8,0,0,19970210,20190409,0,"RUA JOSE LOURENCO, 531, MEIRELES, FORTALEZA, C...",-3.730451,-38.521797
2,0.0,9430800,1679604964,3999,1,71,ASSOCIACAO SANTA EDWIRGES,0,5,16,ASSOCIACAO DE CONFECCIONISTAS DE FORTALEZA DO ...,8,0,0,19970409,20081231,0,"RUA VIA FERREA SOBRAL, 5381, ANTONIO BEZERRA, ...",-3.730451,-38.521797
3,0.0,6911701,1688604964,2232,1,0,0,0,5,49,ROCHA ARAUJO E ARRAIS ADVOGADOS ASSOCIADOS,2,0,0,19970418,20051103,0,"AVENIDA SANTOS DUMONT, 1687, ALDEOTA, FORTALEZ...",-3.736211,-38.498325
4,0.0,4615000,1790604981,2062,1,1,MOVEIS FLORENSE,0,1,49,FLORENSE NORDESTE SERVICOS E REPRESENTACOES LTDA,8,32641323,0,19970429,20091120,0,"RUA EDUARDO SALGADO, 443, ALDEOTA, FORTALEZA, ...",-3.734004,-38.508760


# Processo de filtragem

Abaixo, você poderá realizar o processo de filtragem! :)

O dado fornecido possui os seguintes atributos mais relevantes (na minha humilde opnião) que poderá ser utilizado para melhor compreensão e visualização dos dados! 


*   Bairro
> CORRESPONDE AO BAIRRO ONDE SE LOCALIZA
O ESTABELECIMENTO.
* CEP
> CÓDIGO DE ENDEREÇAMENTO POSTAL
REFERENTE AO LOGRADOURO NO QUAL O
ESTABELECIMENTO ESTA LOCALIZADO.
* Logradouro
> CORRESPONDE AO NOME DO LOGRADOURO
ONDE SE LOCALIZA O ESTABELECIMENTO.
* Complemento
> CORRESPONDE AO COMPLEMENTO PARA O
ENDEREÇO DE LOCALIZAÇÃO DO
ESTABELECIMENTO.
* CNAE fiscal
> INDICA O CÓDIGO DA ATIVIDADE ECONÔMICA .
* Data de inicio de atividade
> DATA DE INICIO DA ATIVIDADE.
* Nome fantasia
> CORRESPONDE AO NOME FANTASIA.
* CPNJ
> CONTEM O NÚMERO DE INSCRIÇÃO NO CNPJ
(CADASTRO NACIONAL DA PESSOA JURÍDICA).
* CNAE fiscal
> INDICA O CÓDIGO DA ATIVIDADE ECONÔMICA.
* Capital social
> CAPITAL SOCIAL DA EMPRESA.
* Porte
> CÓDIGO DO PORTE DA EMPRESA
00 - NAO INFORMADO
01 - MICRO EMPRESA
03 - EMPRESA DE PEQUENO
PORTE
05 - DEMAIS
* Código natureza juridica
> CÓDIGO DA NATUREZA JURÍDICA.
* Matriz filial
> 1 – MATRIZ
2 – FILIAL
* Motivo situação
> CÓDIGO DO MOTIVO DA SITUAÇÃO CADASTRAL.
* Razão social
> CORRESPONDE AO NOME EMPRESARIAL DA PESSOA
JURÍDICA
* Situação 
> 2 DIGITOS
CÓDIGO DA SITUAÇÃO CADASTRAL
01 - NULA
02 - ATIVA
03 - SUSPENSA
04 - INAPTA
08 - BAIXADA


In [0]:
pd.set_option('display.max_rows', None)

Valores únicos para bairro:

In [0]:
print(list(data['bairro'].unique()))

Valores únicos para CEP

In [0]:
print(list(data['cep'].unique()))

Valores únicos para logradouro:

In [0]:
print(list(data['logradouro'].unique()))

Valores únicos para complemento:

In [0]:
print(list(data['complemento'].unique()))

Valores únicos para data_inicio_atividade:

In [0]:
print(list(data['data_inicio_ativ'].unique()))

Valores únicos para capital social:

In [0]:
print(list(data['capital_social'].unique()))

Valores únicos para porte:


In [0]:
print(list(data['porte'].unique()))

Valores únicos para Codigo_natureza_juridica:

In [0]:
print(list(data['cod_nat_juridica'].unique()))

Valores únicos para matriz_filial:

In [0]:
print(list(data['matriz_filial'].unique()))

Valores únicos para motivo_situacao:

In [0]:
print(list(data['motivo_situacao'].unique()))

Valores únicos para situacao:

In [0]:
print(list(data['situacao'].unique()))

# Visualização dos dados

In [12]:
data = data.loc[0:data[data['lat'] == 0].index[0]]
bairro = None
cep = None 
logradouro = None 
complemento = None 


data_inicio_atividade = None
primeira_letra_nome_empresa = None
nome_empresa = None

cnpj = None
cnae_fiscal = None 
capital_social = None
porte = None

codigo_natureza_juridica = None
matriz_filial = None
motivo_situacao = None
razao_social = None
situacao = None


data_ = filter_by(data, bairro, cep, logradouro, complemento, 
          cnae_fiscal, data_inicio_atividade, 
          primeira_letra_nome_empresa, nome_empresa,
          cnpj, capital_social, porte,
          codigo_natureza_juridica, matriz_filial,
          motivo_situacao, razao_social, situacao)

data_.to_html('/content/drive/My Drive/CNPJ/page_with_data.html')
data_

Quantidade de dados:  501


,capital_social,cnae_fiscal,cnpj,cod_nat_juridica,matriz_filial,motivo_situacao,nome_fantasia,opc_simples,porte,qualif_resp,razao_social,situacao,telefone_1,telefone_2,data_inicio_ativ,data_situacao,email,endereco,lat,lon
0,0.0,9430800,508605031,3999,1,71,0,0,5,16,ASSOCIACAO DOS ROTARIANOS DO ROTARY CLUB FORTA...,8,0,0,19970225,20081231,0,"RUA SENADOR POMPEU, 1539, CENTRO, FORTALEZA, C...",-3.733384,-38.531544
1,200000.0,8610101,991604972,2240,1,1,0,0,1,49,AMH ASSISTENCIA MEDICO HOSPITALAR S/C LTDA,8,0,0,19970210,20190409,0,"RUA JOSE LOURENCO, 531, MEIRELES, FORTALEZA, C...",-3.730451,-38.521797
2,0.0,9430800,1679604964,3999,1,71,ASSOCIACAO SANTA EDWIRGES,0,5,16,ASSOCIACAO DE CONFECCIONISTAS DE FORTALEZA DO ...,8,0,0,19970409,20081231,0,"RUA VIA FERREA SOBRAL, 5381, ANTONIO BEZERRA, ...",-3.730451,-38.521797
3,0.0,6911701,1688604964,2232,1,0,0,0,5,49,ROCHA ARAUJO E ARRAIS ADVOGADOS ASSOCIADOS,2,0,0,19970418,20051103,0,"AVENIDA SANTOS DUMONT, 1687, ALDEOTA, FORTALEZ...",-3.736211,-38.498325
4,0.0,4615000,1790604981,2062,1,1,MOVEIS FLORENSE,0,1,49,FLORENSE NORDESTE SERVICOS E REPRESENTACOES LTDA,8,32641323,0,19970429,20091120,0,"RUA EDUARDO SALGADO, 443, ALDEOTA, FORTALEZA, ...",-3.734004,-38.508760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.0,4634601,-163067886,2305,2,0,FRIGORIFICO ACAILANDIA,6,1,65,DISTRIBUIDORA DE ALIMENTOS CRUZEIRO DO SUL EIRELI,2,0,0,19981229,20051103,0,"RUA PADRE PERDIGAO SAMPAIO, 793, ANTONIO BEZER...",-3.739781,-38.594666
497,0.0,4752100,232932091,2135,1,1,KIT GRILL RESTO E BAR,0,5,50,LUIZ REGINALDO GUIMARAES ROCHA,8,0,0,19970516,20020225,0,"RUA BARBOSA DE FREITAS, 100, MEIRELES, FORTALE...",-3.729668,-38.495125
498,0.0,4763601,238932038,2062,1,71,PLANEP IMPORT,0,1,49,SANMA PRESENTES E DECORACOES LTDA,8,0,0,19970516,20081231,0,"AVENIDA GODOFREDO MACIEL, 904, PARANGABA, FORT...",-3.784215,-38.566322
499,0.0,4772500,365932045,2062,1,63,LOJAO C P,0,5,49,IMPERIO DO COSMETICO LTDA,4,0,0,19970516,20181026,0,"RUA 24 DE MAIO, 220, CENTRO, FORTALEZA, CEARA",-3.722398,-38.530500


In [15]:
map = cluster(data_, lat_origin=-3.71839, lon_origin=-38.5434)
map.save(outfile='/content/drive/My Drive/CNPJ/page_with_visualization_map.html')
map